In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import re
import csv


import matplotlib.colors as mc
import colorsys

In [ ]:
#tsne_folder='./'
perplexity=150
#tsne_folder='../../embeddings/Scores/weights_2_150_voiced_unvoiced_bis_2/dropout_20_Hellinger'
tsne_folder='../../embeddings/Scores/weights_2_180_F0/dropout_0_Hellinger'  #SI
#'../../embeddings/Scores/weights_2_150_voiced_unvoiced_ter/dropout_10_Hellinger/'


Import labels

In [ ]:
#predicted
ind_lang=0
#map lang -> ind
dic_lang={}
#map lang -> array line
dic_lang_predicted={}
lang_predicted=[]
ind_lang_predicted=[]
with open(f'{tsne_folder}/labels_predicted.tsv', 'r') as f:
    reader=csv.reader(f)
    for i, line in enumerate(reader):
        lang=line[0]
        if not lang in dic_lang:
            dic_lang[lang]=ind_lang
            dic_lang_predicted[lang]=[]
            ind_lang+=1
        ind_lang_predicted.append(dic_lang[lang])
        lang_predicted.append(lang)
        dic_lang_predicted[lang].append(i)
        
        
            
        

In [ ]:
#true
ind_lang=0
lang_true=[]
ind_lang_true=[]

#map lang -> array line
dic_lang_true={}
for lang in dic_lang_predicted:
    dic_lang_true[lang]=[]
with open(f'{tsne_folder}/labels_true.tsv', 'r') as f:
    reader=csv.reader(f)
    for line in reader:
        lang=line[0]
        if not lang in dic_lang:
            dic_lang[lang]=ind_lang
            
            dic_lang_true[lang]=[]
            ind_lang+=1
        ind_lang_true.append(dic_lang[lang])
        lang_true.append(lang)
        dic_lang_true[lang].append(i)
        

Import coords

Custom t-SNE instead

In [ ]:
from sklearn.manifold import TSNE

use_custom=True

In [ ]:
coords0=[]
with open(f'{tsne_folder}/data.csv', 'r') as f:
    csvreader=csv.reader(f, delimiter='\t')
    next(csvreader)
    for line in csvreader:
        arr=[float(cell) for cell in line[2::]]
        coords0.append(arr)
        
coords0_arr=np.array(coords0)
    

In [ ]:
MyTSNE=TSNE(n_components=2, perplexity=perplexity, learning_rate=100.0, init='random', random_state=2, method='barnes_hut')  #, angle=0.2) 
                     
#other params:, early_exaggeration=12.0, learning_rate=200.0, n_iter=1000
# n_iter_without_progress=300, min_grad_norm=1e-07, metric='euclidean', init='random', verbose=0, 
# method='barnes_hut' or 'exact', angle=0.5, n_jobs=None, square_distances='legacy'

#' Angle less than 0.2 has quickly increasing computation time and angle greater 0.8 has quickly increasing error.
if use_custom:
    coords_arr=MyTSNE.fit_transform(coords0_arr)

Compute centroids, etc.

In [ ]:
centroids=[]
for lang, inds in dic_lang_predicted.items():
    ind_lang=dic_lang[lang]
    inds=np.array(inds)
    coords_inds=coords_arr[inds]
    centroid=np.mean(coords_inds, axis=0)
    centroids.append(centroid)

Plots

In [ ]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

In [ ]:
pl.figure().add_axes((0,0,1,1))
permut=np.random.permutation(len(dic_lang_predicted.keys()))
for lang, inds in dic_lang_predicted.items():
    ind_lang=dic_lang[lang]
    inds=np.array(inds)
    coords_inds=coords_arr[inds]
    pl.plot(coords_inds[:, 0], coords_inds[:, 1], 'o', color=lighten_color(f'C{permut[ind_lang]}'), markersize=4)
    #pl.plot(centroids[ind_lang][0], centroids[ind_lang][1], 'o', alpha=0.40, markersize=8, color='black')
    #pl.text(centroids[ind_lang][0], centroids[ind_lang][1], lang, horizontalalignment='center')
    centroid=centroids[ind_lang]
    d2=(coords_inds[:,0]-centroid[0])**2+(coords_inds[:,1]-centroid[1])**2
    ind_sort=np.argsort(d2)
    
    max_ind_th=round(len(ind_sort)*2/3)
    ind_th=ind_sort[0:max_ind_th]
    pl.plot(coords_inds[ind_th, 0], coords_inds[ind_th, 1], 'o', alpha=0.8, color=f'C{permut[ind_lang]}', markersize=4)
    centroid2=np.mean(coords_inds[ind_th], axis=0)
    
    pl.text(centroid2[0], centroid2[1], lang, horizontalalignment='center', 
            verticalalignment='center', bbox=dict(facecolor='white', alpha=0.5))

    
    pl.savefig('scatterplot.svg')

In [ ]:
np.shape(coords_arr)